Different trial and error methods that we tried during preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Fire_Incidents.tsv', sep='\t')

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
len(df.columns)

In [ ]:
df = df.drop(['Incident Number','Address','Call Number', 'Arrival DtTm', 'Close DtTm', 'Suppression Units', 'EMS Units', 'EMS Personnel', 'Other Units', 'Other Personnel', 'First Unit On Scene', 'Estimated Property Loss', 
         'Estimated Contents Loss', 'Fire Fatalities', 'Fire Injuries', 'Civilian Fatalities', 'Civilian Injuries', 
         'Mutual Aid', 'Action Taken Secondary', 'Action Taken Other', 'Area of Fire Origin', 
         'Ignition Factor Primary', 'Ignition Factor Secondary', 'Heat Source','Item First Ignited',
         'Human Factors Associated with Ignition', 'Floor of Fire Origin', 'Fire Spread','No Flame Spead',
         'Number of floors with minimum damage','Number of floors with significant damage','Number of floors with heavy damage',
         'Number of floors with extreme damage','Detector Type','Detector Operation','Detector Effectiveness',
         'Detector Failure Reason','Automatic Extinguishing System Present','Automatic Extinguishing Sytem Type',
         'Automatic Extinguishing Sytem Perfomance','Automatic Extinguishing Sytem Failure Reason','Location'], axis=1)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
# remove columns with more than half null values
def delete_columns(df, col):
    if df[col].isnull().sum() > df[col].count()/2:
        del df[col]
        
for col in df.columns:
    delete_columns(df, col)

In [ ]:
df.columns

In [ ]:
df_X = df.interpolate(method ='nearest')

In [ ]:
df.shape

In [ ]:
def Property_Use_Conv(df):
    temp = str(df)
    temp = temp.strip()
    #temp = re.sub(r'\d+',temp )
    return temp[:3]

In [ ]:
df_X['Property Use'] = df_X['Property Use'].apply(lambda s: Property_Use_Conv(s))
df_X['Property Use'] = df_X['Property Use'].apply(lambda s: 500 if str(s) == 'uuu' or str(s) == 'nnn' or str(s) == 'nan' or str(s) == '-' else s)
df_X['Property Use'] = df_X['Property Use'].apply(lambda s: str(s)[0] if str(s).find('-') != -1 else s)
df_X['Primary Situation'] = df_X['Primary Situation'].apply(lambda s: Property_Use_Conv(s))
df_X['Primary Situation'] = df_X['Primary Situation'].apply(lambda s: 650 if str(s) == 'n/a' or str(s).startswith('cr') or str(s).startswith('y') or str(s) == '-' else s)
df_X['Primary Situation'] = df_X['Primary Situation'].apply(lambda s: str(s)[0] if str(s).find('-') != -1 or str(s).find('*') != -1 else s)  
df_X['Action Taken Primary'] = df_X['Action Taken Primary'].apply(lambda s: Property_Use_Conv(s))
df_X['Action Taken Primary'] = df_X['Action Taken Primary'].apply(lambda s: 86 if str(s) == '-' or str(s).startswith('Nan') or str(s).startswith('nan') else s)
df_X['Detector Alerted Occupants'] = df_X['Detector Alerted Occupants'].apply(lambda s: Property_Use_Conv(s))
df_X['Detector Alerted Occupants'] = df_X['Detector Alerted Occupants'].apply(lambda s: 0 if str(s) == '-' or str(s).startswith('u') or str(s).startswith('nan') else s)
df_X['Station Area'] = df_X['Station Area'].apply(lambda s: 40 if str(s).startswith('A') or str(s).startswith('H') or str(s).startswith('O') or str(s).startswith('nan') or str(s) == '-' else s)
df_X['Detector Alerted Occupants'] = df_X['Detector Alerted Occupants'].apply(lambda s: str(s)[0] if str(s).find('-') != -1 or str(s).find('*') != -1 or str(s).find('d') != -1 else s)  

In [ ]:
df_X.shape

In [ ]:
from datetime import datetime

def convert_date_to_weekday(tmstp):
    result = datetime.strptime(tmstp, '%m/%d/%Y %I:%M:%S %p').weekday()
    return result
    
def convert_date_to_hour(tmstp):
    result = datetime.strptime(tmstp, '%m/%d/%Y %I:%M:%S %p').hour
    return result

def convert_date_to_month(tmstp):
    result = datetime.strptime(tmstp, '%m/%d/%Y %I:%M:%S %p').month
    return result

df_X['Weekday'] = df_X['Alarm DtTm'].apply(lambda x: convert_date_to_weekday(x))
df_X['Hour'] = df_X['Alarm DtTm'].apply(lambda x: convert_date_to_hour(x))
df_X['Month'] = df_X['Alarm DtTm'].apply(lambda x: convert_date_to_month(x))

In [ ]:
# df_Y = df_X['Suppression Personnel']

In [ ]:
# df_Y.value_counts()

In [ ]:
def missing_values():
    temp_dict = dict()
    for i in df_X.columns:
        if df_X[i].isnull().sum() > 0: 
            temp_dict[i] = df_X[i].isnull().sum()
    return temp_dict

In [ ]:

missing_values()

for col in df_X.columns:
    df_X[col] = df_X[col].fillna(method='bfill')

In [ ]:
df_X['Battalion'] = pd.Categorical(df_X['Battalion'])
one_hot = pd.get_dummies(df_X['Battalion'],prefix='Battalion')
    #df_X2 = df_X2.drop('Battalion',axis = 1)
    # Join the encoded df
df_X = df_X.join(one_hot)
df_X['Zipcode'] = pd.Categorical(df_X['Zipcode'])
one_hot = pd.get_dummies(df_X['Zipcode'],prefix='Zipcode')
    #df_X2 = df_X2.drop('Battalion',axis = 1)
    # Join the encoded df
df_X = df_X.join(one_hot)
df_feature  = df_X.drop(['Battalion','Zipcode'],axis=1)


In [ ]:
df_feature = df_feature.drop(['Alarm DtTm','City','Incident Date','Neighborhood  District'],axis=1)

In [ ]:
df_feature.shape

In [ ]:
filter_df = df_feature[df_feature['Suppression Personnel'] <= 100]

In [ ]:
filter_df.shape

In [ ]:
filter_df['Suppression Personnel'].value_counts()

In [ ]:
df_Y = filter_df['Suppression Personnel']

In [ ]:
filter_df = filter_df.drop(['Suppression Personnel'],axis=1)


In [ ]:
# df_X1_mini = df_feature[:50000]
# df_y_mini = df_Y[:50000]

In [ ]:
df_Y.shape

In [ ]:
from scipy.sparse import csr_matrix


In [ ]:
def dataframetoCSRmatrix(df):
    nrows = len(df)
    nc = len(df.columns)
    idx = {}
    tid = 0
    nnz = nc * nrows
    
    cols= df.columns
    
    for col in cols:
        df[col] = df[col].apply(str)
        for name in df[col].unique():
            idx[col+name] = tid
            tid += 1
    
    ncols = len(idx)
    
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.int)
    
    i=0
    n=0
    
    for index,row in df.iterrows():
        for j,col in enumerate(cols):
            ind[j+n] = idx[col+row[col]]
            val[j+n] = 1
        ptr[i+1] = ptr[i] + nc
        n += nc
        i += 1
    
    mat = csr_matrix((val,ind,ptr), shape=(nrows,ncols), dtype=np.int)
    mat.sort_indices()   
    
    return mat

In [ ]:
mat1 = dataframetoCSRmatrix(filter_df)


In [ ]:
# from sklearn.decomposition import TruncatedSVD

# svd = TruncatedSVD(n_components=30, n_iter=10, random_state=42)
# reduced_mat = svd.fit_transform(mat1)

In [ ]:
# reduced_mat.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error

In [ ]:
df_feature_train, df_feature_test, df_y_train, df_y_test = train_test_split(mat1, df_Y, test_size = 0.2, random_state = 0)


# rfreg = RandomForestRegressor(n_estimators= 10,max_depth=1, random_state=42)
# rfreg.fit(df_feature_train, df_y_train)


# predicted_y_rf = rfreg.predict(df_feature_test)


# set(predicted_y_rf)



In [ ]:
# gbr = GradientBoostingRegressor(n_estimators=175, learning_rate=0.08, max_depth=3, random_state=1232, loss='ls')

In [ ]:
# gbr.fit(x_train, y_train)

In [ ]:
# predicted_y_rf = rfreg.predict(df_feature_test)


In [ ]:
# mat1.shape

In [ ]:
# df_Y.values.shape

In [ ]:

# rfreg = RandomForestRegressor(n_estimators= 10,max_depth=1, random_state=42)
# rfreg.fit(df_feature_train, df_y_train)



In [ ]:
# predicted_y_rf = rfreg.predict(df_feature_test)


In [ ]:
# set(predicted_y_rf)




In [ ]:
# from sklearn.decomposition import TruncatedSVD

# svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
# reduced_mat = svd.fit_transform(mat1)

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
reg_Ridge = Ridge(alpha = 0.000001)

In [ ]:
reg_Ridge.fit(df_feature_train, df_y_train)


In [ ]:
y_pred_list_Ridge = reg_Ridge.predict(df_feature_test)


In [ ]:
# set(y_pred_list_Ridge)

In [ ]:
print("r2_score:_ Ridge",r2_score(df_y_test, y_pred_list_Ridge))
print("explained_variance_score:Ridge ",explained_variance_score(df_y_test, y_pred_list_Ridge))
print("mean_absolute_error:Ridge ",mean_absolute_error(df_y_test, y_pred_list_Ridge))
print("mean_squared_error:Ridge ",mean_squared_error(df_y_test, y_pred_list_Ridge))
print("median_absolute_error:Ridge ",median_absolute_error(df_y_test, y_pred_list_Ridge))
# print("mean_squared_log_error:Ridge ",mean_squared_log_error(df_y_test, y_pred_list))


In [ ]:
# alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]



In [ ]:

# print("r2_score:_ GradientDescent",r2_score(df_y_test, predicted_y_rf))
# print("explained_variance_score:_GradientDescent ",explained_variance_score(df_y_test, predicted_y_rf))
# print("mean_absolute_error:_GradientDescent ",mean_absolute_error(df_y_test, predicted_y_rf))
# print("mean_squared_error:_GradientDescent ",mean_squared_error(df_y_test, predicted_y_rf))
# print("median_absolute_error:_GradientDescent ",median_absolute_error(df_y_test, predicted_y_rf))
# print("mean_squared_log_error:_GradientDescent ",mean_squared_log_error(df_y_test, predicted_y_rf))


In [ ]:
from sklearn.externals import joblib 


In [ ]:
joblib.dump(reg_Ridge, 'sf_reg_ridge.pkl') 